### 주제: 5대 리그 (PL, LaLiga, Serie A, Bundesliga, Ligue 1) 내 이적시장 목록 크롤링 및 자동화

In [1]:
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
    }

In [2]:
league_code = ['GB1', 'ES1', 'IT1', 'L1', 'FR1']    # PL, LaLiga, Serie A, Bundesliga, Ligue 1
name = []
position = []
age = []
nation = []
left = []
joined = []
transfer_date = []
market_value = []
fee = []

for league in league_code:
    url = f'https://www.transfermarkt.com/transfers/neuestetransfers/statistik/plus/plus/1/galerie/0/wettbewerb_id/{league}/land_id//minMarktwert/500.000/maxMarktwert/500.000.000/minAbloese/0/maxAbloese/500.000.000/yt0/Show/page/1?ajax=yw1'
    response = requests.get(url, headers=headers)
    # print(response.text)
    soup = BeautifulSoup(response.text, 'html.parser')
    # transfer_lists = soup.find_all('tr',{'class':['odd','even']})
    transfer_lists = soup.select('tr.odd, tr.even')
    # print(transfer_lists)
    
    for transfer_list in transfer_lists:
        info = transfer_list.select('td')
        # for i in enumerate(info):
        #     print(i)
        name.append(info[2].text.strip())
        position.append(info[3].text.strip())
        age.append(info[4].text.strip())
        nation.append(info[5].img['alt'])
        left.append(info[6].img['alt'])
        joined.append(info[10].img['alt'])
        transfer_date.append(info[14].text.strip())
        market_value.append(info[15].text.strip())
        fee.append(info[16].text.strip())


In [3]:
print(name)
print(position)
print(age)
print(nation)
print(left)
print(joined)
print(transfer_date)
print(market_value)
print(fee)

['Thomas Kaminski', 'Matija Sarkic', 'Auston Trusty', 'Richie Laryea', 'Manuel Lanzini', 'Luke McNally', 'Aaron Connolly', 'Deniz Undav', 'Lucas Moura', 'Braian Ojeda', 'Josh Bowler', 'Lesley Ugochukwu', 'Iliman Ndiaye', 'Luke Plange', 'Jack Colback', 'Fabinho', 'Carl Rushworth', 'Allan Saint-Maximin', 'Karl Darlow', 'Wesley Moraes', 'Ryan Giles', 'Calvin Bassey', 'Fred Onyedinma', 'Riyad Mahrez', 'Jamal Lewis', 'Rodrigo Ely', 'Cyle Larin', 'Rony Lopes', 'Mario Hernández', 'André Silva', 'Asier Illarramendi', 'Erick Cabaco', 'Julián Araujo', 'Diego Lainez', 'Álex Collado', 'Jorge Meré', 'Florin Andone', 'Rubén Blanco', 'Karim Rekik', 'Munir El Haddadi', 'Nico González', 'Edinson Cavani', 'El Bilal Touré', 'Florian Lejeune', 'Rafa Marín', 'Gerard Gumbau', 'Samuel Chukwueze', 'Álex Collado', 'Chadi Riad', 'Johan Mojica', 'Gino Infantino', 'Luca Vido', "Danilo D'Ambrosio", 'Mert Müldür', 'Miguel Veloso', 'Morten Thorsby', 'Magnus Warming', 'Marco Carraro', 'Giuseppe Ambrosino', 'Stefano G

In [72]:
df = pd.DataFrame(
    {'Name': name,
    'Position': position,
    'Age': age,
    'Nation': nation,
    'Left': left,
    'Joined': joined,
    'Transfer Date': transfer_date,
    'Market Value': market_value,
    'Fee': fee
})

df

,Name,Position,Age,Nation,Left,Joined,Transfer Date,Market Value,Fee
0,Thomas Kaminski,Goalkeeper,30,Belgium,Blackburn Rovers,Luton Town,"Aug 3, 2023",€2.20m,€2.90m
1,Matija Sarkic,Goalkeeper,26,Montenegro,Wolverhampton Wanderers,Millwall FC,"Aug 3, 2023",€2.00m,€1.40m
2,Auston Trusty,Centre-Back,24,United States,Arsenal FC,Sheffield United,"Aug 3, 2023",€10.00m,€5.80m
3,Richie Laryea,Right-Back,28,Canada,Nottingham Forest,Vancouver Whitecaps FC,"Aug 2, 2023",€2.50m,loan transfer
4,Manuel Lanzini,Attacking Midfield,30,Argentina,West Ham United,CA River Plate,"Aug 2, 2023",€6.00m,free transfer
...,...,...,...,...,...,...,...,...,...
120,Mauro Icardi,Centre-Forward,30,Argentina,Paris Saint-Germain,Galatasaray,"Jul 30, 2023",€19.00m,€10.00m
121,Cem Türkmen,Central Midfield,21,Turkey,Clermont Foot 63,MKE Ankaragücü,"Jul 28, 2023",€500k,?
122,Dario Maresic,Centre-Back,23,Austria,Stade Reims,NK Istra 1961,"Jul 27, 2023",€500k,?
123,Papa Ndiaga Yade,Left Winger,23,Senegal,FC Metz,Quevilly Rouen Métropole,"Jul 27, 2023",€1.20m,loan transfer


### 데이터 전처리

In [73]:
# 5대 리그 사이에서 이적이 발생할 경우, 중복 발생하므로 중복 제거
df.drop_duplicates(['Name'], inplace=True)
df.reset_index(inplace=True, drop=True)

In [74]:
# Age 컬럼 정수형으로 변환
df['Age'] = df['Age'].astype('int64')

In [75]:
# 날짜 형식 변환 및 날짜 순으로 정렬
df['Transfer Date'] = pd.to_datetime(df['Transfer Date'])
df.sort_values(by='Transfer Date', ascending=False, inplace=True)

In [80]:
# 시장 가치 데이터 변환
df['Market Value'] = df['Market Value'].apply(lambda x: re.search('[0-9\.]{1,}[mk]', x).group())
df['Market Value'] = df['Market Value'].apply(lambda x: float(str(x)[:-1])*10e6 if str(x)[-1] == 'm' else float(str(x)[:-1])*10e3)

In [81]:
# 임대/이적 구분
df['Transfer Type'] = df['Fee'].apply(lambda x: 'loan' if re.search('[lL]oan.+', x) else 'unknown' if x == 'unknown' else 'transfer')

In [82]:
# 비용 데이터 변환
df['Fee'] = df['Fee'].apply(lambda x: re.search('[0-9\.]{1,}[mk]', x).group() if re.search('[0-9\.]{1,}[mk]', x) else 'unknown' if x == '?' else 0)
df['Fee'] = df['Fee'].apply(lambda x: float(str(x)[:-1])*10e6 if str(x)[-1] == 'm' else float(str(x)[:-1])*10e3 if str(x)[-1] == 'k' else 'unknown' if str(x) == 'unknown' else 0)

In [83]:
# Market Value, Fee column명 변경
df.rename(columns={'Market Value': 'Market Value (€)', 'Fee': 'Fee (€)'}, inplace=True)

In [84]:
df

,Name,Position,Age,Nation,Left,Joined,Transfer Date,Market Value (€),Fee (€),Transfer Type
0,Thomas Kaminski,Goalkeeper,30,Belgium,Blackburn Rovers,Luton Town,2023-08-03,22000000.0,29000000.0,transfer
25,Rodrigo Ely,Centre-Back,29,Brazil,UD Almería,Grêmio Foot-Ball Porto Alegrense,2023-08-03,30000000.0,15000000.0,transfer
50,Luca Vido,Centre-Forward,26,Italy,Atalanta BC,AC Reggiana 1919,2023-08-03,7000000.0,unknown,transfer
49,Gino Infantino,Central Midfield,20,Argentina,CA Rosario Central,ACF Fiorentina,2023-08-03,30000000.0,40000000.0,transfer
72,Diant Ramaj,Goalkeeper,21,Germany,Eintracht Frankfurt,Ajax Amsterdam,2023-08-03,7500000.0,80000000.0,transfer
...,...,...,...,...,...,...,...,...,...,...
89,Ragnar Ache,Centre-Forward,25,Germany,Eintracht Frankfurt,1.FC Kaiserslautern,2023-07-25,10000000.0,10000000.0,transfer
90,Alexander Nübel,Goalkeeper,26,Germany,Bayern Munich,VfB Stuttgart,2023-07-25,80000000.0,10000000.0,loan
92,Attila Szalai,Centre-Back,25,Hungary,Fenerbahce,TSG 1899 Hoffenheim,2023-07-24,140000000.0,123000000.0,transfer
91,Marcel Sabitzer,Central Midfield,29,Austria,Bayern Munich,Borussia Dortmund,2023-07-24,200000000.0,190000000.0,transfer
